# Neptune as Graph Memory

TODO: what is mem0

TODO: types of mem0 storage [vector storage, graph storage]

References

## Prerequisites

### 1. Install Mem0 with Graph Memory support 

To use Mem0 with Graph Memory support, install it using pip:

```bash
pip install "mem0ai[graph]"
```

This command installs Mem0 along with the necessary dependencies for graph functionality.

### 2. Connect to Neptune

To connect to AWS Neptune Analytics, you need to configure Neptune with your AWS profile details. I have configured AWS credentials in my home profile and can connect by specifying the AWS_PROFILE environment variable

TODO: Reference to: Neptune setup; aws-langchain

### 3. Configure OpenSearch

We're going to use OpenSearch as our vector store.  You can run [OpenSearch from docker image](https://docs.opensearch.org/docs/latest/install-and-configure/install-opensearch/docker/):

```bash
docker pull opensearchproject/opensearch:3
```

And verify that it's running with a `<custom-admin-password>`:

```bash
 docker run -d -p 9200:9200 -p 9600:9600 -e "discovery.type=single-node" -e "OPENSEARCH_INITIAL_ADMIN_PASSWORD=<custom-admin-password>" opensearchproject/opensearch:latest

 curl https://localhost:9200 -ku admin:<custom-admin-password>
```

We're going to connect [OpenSearch using the python client](https://github.com/opensearch-project/opensearch-py):

```bash
pip install "opensearch-py"
```

## Configuration

Do all the imports and configure OpenAI (enter your OpenAI API key):

In [6]:
from mem0 import Memory
from opensearchpy import OpenSearch
from langchain_aws import NeptuneAnalyticsGraph
import os, logging, sys

logging.getLogger('mem0.memory.neptune_memory').setLevel(logging.DEBUG)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

logging.basicConfig(
    format='%(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout  # Explicitly set output to stdout
)

Setup the Mem0 configuration using:
- openai as the embedder
- AWS Neptune Analytics instance as a graph store
- OpenSearch as the vector store

In [7]:
graph_identifier = os.environ.get("GRAPH_ID")
opensearch_username = os.environ.get("OPENSEARCH_USERNAME")
opensearch_password = os.environ.get("OPENSEARCH_PASSWORD")
config = {
    "embedder": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-3-large",
            "embedding_dims": 1536
        },
    },
    "graph_store": {
        "provider": "neptune",
        "config": {
            "graph_identifier": graph_identifier,
            # "endpoint_url": endpoint_url,
            # "region": region,
        },
    },
    "vector_store": {
        "provider": "opensearch",
        "config": {
            "collection_name": "vector_store",
            "host": "localhost",
            "port": 9200,
            "user": opensearch_username,
            "password": opensearch_password,
            "use_ssl": False,
            "verify_certs": False,
        },
    },
}

## Graph Memory initializiation 

Initialize Memgraph as a Graph Memory store: 

In [8]:
m = Memory.from_config(config_dict=config)

app_id = "movies"
user_id = "alice"

## Store memories 

Create memories:

In [9]:
messages = [
    {
        "role": "user",
        "content": "I'm planning to watch a movie tonight. Any recommendations?",
    },
    {
        "role": "assistant",
        "content": "How about a thriller movies? They can be quite engaging.",
    },
    {
        "role": "user",
        "content": "I'm not a big fan of thriller movies but I love sci-fi movies.",
    },
    {
        "role": "assistant",
        "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future.",
    },
]

Store memories in Neptune Analytics:

In [10]:
# Store inferred memories (default behavior)
result = m.add(messages, user_id=user_id, metadata={"category": "movie_recommendations"})

DEBUG - Entity type map: {'movie': 'media', 'thriller': 'genre', 'sci-fi': 'genre', 'i': 'person', 'my': 'possessive_pronoun'}
 search_results={'content': None, 'tool_calls': [{'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'movie', 'entity_type': 'media'}, {'entity': 'thriller', 'entity_type': 'genre'}, {'entity': 'sci-fi', 'entity_type': 'genre'}]}}, {'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'I', 'entity_type': 'person'}, {'entity': 'my', 'entity_type': 'possessive pronoun'}]}}]}
DEBUG - Extracted entities: [{'source': 'alice', 'relationship': 'plans_to_watch', 'destination': 'movie'}, {'source': 'movie', 'relationship': 'is_a', 'destination': 'thriller'}, {'source': 'movie', 'relationship': 'is_a', 'destination': 'sci-fi'}, {'source': 'alice', 'relationship': 'does_not_favor', 'destination': 'thriller'}, {'source': 'alice', 'relationship': 'loves', 'destination': 'sci-fi'}, {'source': 'movie', 'relationship': 'recommended_for', 'destinatio

## Search memories

In [11]:
for result in m.search("what does alice love?", user_id=user_id)["results"]:
    print(f"\"{result["memory"]}\" [score: {result["score"]}]")

DEBUG - Entity type map: {'alice': 'person'}
 search_results={'content': None, 'tool_calls': [{'name': 'extract_entities', 'arguments': {'entities': [{'entity': 'alice', 'entity_type': 'Person'}]}}]}
INFO - _search_graph_db
  query=
            MATCH (n)
            WHERE n.embedding IS NOT NULL AND n.user_id = $user_id
            WITH n, n.embedding AS embedding, $n_embedding as n_embedding
            CALL neptune.algo.vectors.distanceByEmbedding(embedding, n_embedding)
            YIELD distance
            WITH n, round(2 * distance - 1) AS similarity
            WHERE similarity >= $threshold
            CALL {
                WITH n
                MATCH (n)-[r]->(m) 
                RETURN n.name AS source, id(n) AS source_id, type(r) AS relationship, id(r) AS relation_id, m.name AS destination, id(m) AS destination_id
                UNION
                MATCH (m)-[r]->(n) 
                RETURN m.name AS source, id(m) AS source_id, type(r) AS relationship, id(r) AS relation

Neptune Analytics Graph

In [17]:
all_results = m.get_all(user_id=user_id)
for n in all_results['results']:
    print(f"node \"{n["memory"]}\": [hash: {n["hash"]}]")

for e in all_results['relations']:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")

DEBUG - filters={'user_id': 'alice'}
DEBUG - Retrieved 17 relationships
node "Planning to watch a movie tonight": [hash: bf55418607cfdca4afa311b5fd8496bd]
node "Not a big fan of thriller movies": [hash: 028dfab4483f28980e292f62578d3293]
node "Loves sci-fi movies": [hash: 1110b1af77367917ea2022355a16f187]
edge "movie" --is_a_genre--> "sci-fi"
edge "movie" --is_a--> "sci-fi"
edge "movie" --recommended_for--> "alice"
edge "movie" --is_a_genre--> "thriller"
edge "movie" --is_avoided_for--> "thriller"
edge "movie" --is_a--> "thriller"
edge "alice" --will_suggest--> "sci-fi_movies"
edge "alice" --likes--> "sci-fi"
edge "alice" --loves--> "sci-fi"
edge "alice" --plans_to_watch--> "movie"
edge "alice" --will_avoid--> "thriller_recommendations"
edge "alice" --does_not_favor--> "thriller"
edge "alice" --does_not_like--> "thriller"
edge "alice" --plans_to_watch--> "movie"
edge "movie" --is_a--> "sci-fi"
edge "movie" --recommended_for--> "alice"
edge "movie" --is_a--> "thriller"


In [19]:
neptune_graph = NeptuneAnalyticsGraph(graph_identifier)

query = """
        MATCH (n {user_id: $user_id})-[r]->(m {user_id: $user_id})
        RETURN n.name AS source, type(r) AS relationship, m.name AS target
        LIMIT $limit
        """
edge_results = neptune_graph.query(
    query, params={"user_id": user_id, "limit": 100}
)
print("----RELATIONSHIPS----")
for e in edge_results:
    print(f"edge \"{e["source"]}\" --{e["relationship"]}--> \"{e["target"]}\"")


----RELATIONSHIPS----
edge "movie" --is_avoided_for--> "thriller"
edge "movie" --is_a--> "thriller"
edge "movie" --is_a_genre--> "thriller"
edge "movie" --is_a_genre--> "sci-fi"
edge "movie" --recommended_for--> "alice"
edge "movie" --is_a--> "sci-fi"
edge "alice" --plans_to_watch--> "movie"
edge "alice" --will_avoid--> "thriller_recommendations"
edge "alice" --does_not_favor--> "thriller"
edge "alice" --does_not_like--> "thriller"
edge "movie" --is_a--> "thriller"
edge "alice" --plans_to_watch--> "movie"
edge "alice" --loves--> "sci-fi"
edge "movie" --recommended_for--> "alice"
edge "movie" --is_a--> "sci-fi"
edge "alice" --likes--> "sci-fi"
edge "alice" --will_suggest--> "sci-fi_movies"


In [32]:
opensearch_index = "vector_store"
vector_store_config = config[opensearch_index]["config"]

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': vector_store_config["host"], 'port': vector_store_config["port"]}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (vector_store_config["user"], vector_store_config["password"]),
    use_ssl = vector_store_config["use_ssl"],
    verify_certs = vector_store_config["verify_certs"],
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

query = {"query":{"match_all": {}}}

response = client.search(
    body = query,
    index = opensearch_index,
)

print("----VECTORS----")
for v in response["hits"]["hits"]:
    print(f"vector id=\"{v["_source"]["id"]}\" data={v["_source"]["payload"]["data"]}")

----VECTORS----
vector id="54568998-76a2-4654-9a4b-84c6918d5145" data=Planning to watch a movie tonight
vector id="2160548e-e77e-4c5d-a6cd-e4d6c9d1db9e" data=Not a big fan of thriller movies
vector id="d2137d32-11f7-4d90-8e50-277e5f6014f4" data=Loves sci-fi movies
vector id="203ce467-0bbe-4248-8fd5-9631eb6bbb88" data=User wants to know about famous people from India
vector id="a88da4b9-75c7-4630-be13-13030ee28cb4" data=Interested in famous Indian dishes to try
vector id="62493c28-f9d5-45b9-987e-381a5cce8940" data=Enjoys spicy food, particularly vindaloo
vector id="05e7a57e-52ed-4493-9d93-446c1026de2b" data=Tried vindaloo in Mumbai during a business trip last year
vector id="04c7fae3-21db-4907-9a4e-c5d52a35bb2d" data=Enjoys spicy food, particularly vindaloo
vector id="b5ad9bd9-b136-4d21-8278-ca41a5b3b993" data=Tried vindaloo in Mumbai during a business trip last year
